In [ ]:
import os
import requests
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import exposure
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
import tensorflow as tf

# Step 1: Download and extract the dataset
def download_and_extract(url, extract_to='data'):
    # Download the file
    response = requests.get(url)
    zip_file_path = os.path.join(extract_to, 'Data.zip')
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Download the data
data_url = 'https://dicom5c.blob.core.windows.net/public/Data.zip'
download_and_extract(data_url)

# Step 2: Load Data
def load_data(data_path):
    images = []
    masks = []
    for folder in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            if file.endswith('.tif'):  # Load only TIFF files
                if 'mask' in file:
                    mask = plt.imread(os.path.join(folder_path, file))
                    masks.append(mask)
                else:
                    image = plt.imread(os.path.join(folder_path, file))
                    # Convert image to grayscale if it has 3 channels
                    if image.ndim == 3 and image.shape[2] == 3:  # RGB image
                        image = rgb2gray(image)
                    images.append(image)
    return np.array(images), np.array(masks)

# Load Dataset
data_path = 'data/Data'  # Path to the extracted data folder
images, masks = load_data(data_path)

# Print sizes
print(f'Total images found: {len(images)}')
print(f'Total masks found: {len(masks)}')

# Step 3: Preprocess Data
def preprocess_data(images, masks):
    # Apply CLAHE to images
    clahe = exposure.equalize_adapthist(images, clip_limit=0.03)
    images = clahe.reshape(images.shape)

    # Normalize images and masks
    images = images.astype(np.float32) / np.max(images)
    masks = masks.astype(np.float32) / np.max(masks)

    return images, masks

# Preprocess data
images, masks = preprocess_data(images, masks)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Define input shape
input_shape = (X_train.shape[1], X_train.shape[2], 1)  # Grayscale images

# Step 4: Define Nested U-Net Architecture
def nested_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Step 5: Train Nested U-Net
nested_model = nested_unet(input_shape)
nested_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nested_history = nested_model.fit(X_train[..., np.newaxis], y_train[..., np.newaxis],
                                   validation_data=(X_test[..., np.newaxis], y_test[..., np.newaxis]),
                                   epochs=50, batch_size=16)

# Step 6: Evaluate Model
def evaluate_model(model, X_test, y_test):
    # Calculate DICE score
    predictions = model.predict(X_test)
    predictions = (predictions > 0.5).astype(np.float32)  # Binarize predictions
    dice_score = 2 * np.sum(predictions * y_test) / (np.sum(predictions) + np.sum(y_test) + 1e-6)
    return dice_score

average_dice = evaluate_model(nested_model, X_test[..., np.newaxis], y_test[..., np.newaxis])
print(f'Average DICE Score: {average_dice:.4f}')

# Step 7: Visualize Results
def visualize_predictions(X_test, y_test, predictions, num_images=5):
    for i in range(num_images):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.title("Original Image")
        plt.imshow(X_test[i], cmap='gray')
        plt.subplot(1, 3, 2)
        plt.title("Ground Truth Mask")
        plt.imshow(y_test[i], cmap='gray')
        plt.subplot(1, 3, 3)
        plt.title("Predicted Mask")
        plt.imshow(predictions[i, ..., 0], cmap='gray')
        plt.show()

predictions = nested_model.predict(X_test[..., np.newaxis])
visualize_predictions(X_test, y_test, predictions)

# Optional: Save the model
nested_model.save('nested_unet_model.h5')


FileNotFoundError: [Errno 2] No such file or directory: 'data/Data.zip'